In [ ]:
import json 
import pandas as pd
import torch
from sentence_transformers import SentenceTransformer, CrossEncoder
import pickle
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
data_path = '/data/Eli5/Eli5_reranked/eli5_reranked.json'

In [ ]:
# read dataset
with open(data_path, 'r') as f:
    data = json.load(f)

data = pd.read_json(data, orient='records')

In [ ]:
# format of title differs between eli5 & wow datasets
# passage titles only needed when using facebook dpr encoder

passage_titles = []
if 'output' in data.columns:
    for i in range(len(data)):
        title = data['output'][i][0]['provenance'][0]['title']
        passage_titles.append(title)
else:
    for i in range(len(data)):
        title = data['passages'][i][0]['title']
        passage_titles.append(title)

In [ ]:
model = 'msmarco-distilbert-base-tas-b'
bi_encoder = SentenceTransformer(model)

In [ ]:
# passage format for DPR context encoder needs 'title [SEP] passage' as format

if 'dpr' in model:
    passages = []
    for i in range(0,len(data)):
        passages.append(passage_titles[i]+ ' [SEP] ' + data['passages_text'][i])
else:
    passages = list(data['passages_text'])

In [ ]:
corpus_embeddings = bi_encoder.encode(passages, convert_to_tensor=True, show_progress_bar=True)

Batches:   0%|          | 0/1794 [00:00<?, ?it/s]

In [ ]:
# save corpus embeddings 
output_path = '/contextretrieval/bi-encoder/wow/embeddings/msmarco-distilbert-base-tas-b.pickle'
with open(output_path, 'wb') as pkl:
    pickle.dump(corpus_embeddings, pkl)